In [ ]:
!wget https://github.com/Prayas-Agrawal/tox-bias-data/raw/main/data.zip
!pip install shap



--2023-11-30 01:59:18--  https://github.com/Prayas-Agrawal/tox-bias-data/raw/main/data.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Prayas-Agrawal/tox-bias-data/main/data.zip [following]
--2023-11-30 01:59:18--  https://raw.githubusercontent.com/Prayas-Agrawal/tox-bias-data/main/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99173537 (95M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  94.58M   304MB/s    in 0.3s    

2023-11-30 01:59:20 (304 MB/s) - ‘data.zip’ saved [99173537/99173537]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.

In [ ]:
!unzip /content/data.zip


Archive:  /content/data.zip
  inflating: toxicity_annotated_comments.tsv  
  inflating: toxicity_annotations.tsv  
  inflating: toxicity_worker_demographics.tsv  
  inflating: wiki_dev.csv            
  inflating: wiki_dev_sep.txt        
  inflating: wiki_noised_dev.csv     
  inflating: wiki_noised_test.csv    
  inflating: wiki_noised_train.csv   
  inflating: wiki_test.csv           
  inflating: wiki_test_sep.txt       
  inflating: wiki_train.csv          
  inflating: wiki_train_sep.txt      


In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip


--2023-11-30 01:59:33--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-11-30 01:59:33--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-11-30 01:59:34--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
import numpy as np
import re
import tensorflow.compat.v1 as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb


UNK = '<unk>'
PAD = '<pad>'
TARGET = '<target>'

def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

def load_data_and_labels(data_file, sent_filter=[]):
    """
    Loads data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load data from files
    examples = list(open(data_file, "r", encoding='utf-8').readlines())
    x_text = []
    labels = []
    for i, s in enumerate(examples):
      split = s.strip().split("|||")
      label = split[0]
      text = "".join(split[1:])
      cleaned_str = clean_str(text)
      # Only include words in sent_filter
      if len(sent_filter) > 0 and set(sent_filter).isdisjoint(cleaned_str.split()):
        continue
      x_text.append(clean_str(text))
      labels.append(int(label))
    y = np.array(labels)
    return x_text, y

def get_word_count(corpus):
  word_count = {}
  for sentence in corpus:
    words = sentence.split()
    for word in words:
      if word not in word_count:
        word_count[word] = 0
      word_count[word] += 1
  return word_count

def write_list_to_file(l, filename):
  with open(filename, 'w') as f:
    for item in l:
        f.write("%s\n" % item)

def build_vocab(data_path, min_freq, output_path, target_words=None):
  data = list(open(data_path, "r", encoding='utf-8').readlines())
  text = [clean_str(line.strip().split("|||")[1]) for line in data]
  word_count = get_word_count(text)
  idx2word = [UNK, PAD]
  word2idx = {UNK : 0, PAD : 1}
  if target_words:
    idx2word.append(TARGET)
    word2idx[TARGET] = 3
  for word in word_count:
    if target_words and word in target_words:
      continue
    if word_count[word] >= min_freq:
      idx2word.append(word)
      word2idx[word] = len(idx2word) - 1

  write_list_to_file(idx2word, output_path)
  return idx2word, word2idx

def load_vocab(vocab_file):
  with open(vocab_file) as f:
    lines = f.readlines()
  idx2word = []
  word2idx = {}
  for i, line in enumerate(lines):
    vocab = line.strip()
    idx2word.append(vocab)
    word2idx[vocab] = i
  return idx2word, word2idx

def text2idx(text, word2idx, max_len, target_words=[]):
  ret = []
  for sentence in text:
    words = sentence.split()
    idx = []
    for i, word in enumerate(words):
      if len(idx) >= max_len:
        break
      if word in word2idx:
        idx.append(word2idx[word])
      elif word in target_words:
        idx.append(word2idx[TARGET])
      else:
        idx.append(word2idx[UNK])
    for i in range(len(idx), max_len):
      idx.append(word2idx[PAD])
    ret.append(idx)
  return np.array(ret)

def get_all_pad(length, word2idx):
  return np.array([[word2idx[PAD] for i in range(length)]])

def batch_iter(inputs, outputs, batch_size, num_epochs, shuffle=False, seed=None):
    if seed is not None:
      np.random.seed(seed)
    inputs = np.array(inputs)
    outputs = np.array(outputs)
    num_batches_per_epoch = (len(inputs) - 1) // batch_size + 1
    for epoch in range(num_epochs):
      if shuffle:
        p = np.random.permutation(len(inputs))
        inputs = inputs[p]
        outputs = outputs[p]
      for batch_num in range(num_batches_per_epoch):
          start_index = batch_num * batch_size
          end_index = min((batch_num + 1) * batch_size, len(inputs))
          yield inputs[start_index:end_index], outputs[start_index:end_index]

def batch_iter_attr(inputs, outputs, attr_target, target_words_mask, batch_size, num_epochs, shuffle=False, seed=None):
    if seed is not None:
      np.random.seed(seed)
    inputs = np.array(inputs)
    outputs = np.array(outputs)
    attr_target = np.array(attr_target)
    target_words_mask = np.array(target_words_mask)
    num_batches_per_epoch = (len(inputs) - 1) // batch_size + 1
    for epoch in range(num_epochs):
      if shuffle:
        p = np.random.permutation(len(inputs))
        inputs = inputs[p]
        outputs = outputs[p]
        attr_target = attr_target[p]
        target_words_mask = target_words_mask[p]
      for batch_num in range(num_batches_per_epoch):
          start_index = batch_num * batch_size
          end_index = min((batch_num + 1) * batch_size, len(inputs))
          yield inputs[start_index:end_index], outputs[start_index:end_index], attr_target[start_index:end_index], target_words_mask[start_index:end_index]

def preprocess(data_file, max_len, word2idx, target_words=None, return_attr_target=False, sent_filter=[], target_words_to_token=False):
  x_text, y = load_data_and_labels(data_file, sent_filter)
  if target_words_to_token and target_words:
    x = text2idx(x_text, word2idx, max_len, target_words)
  else:
    x = text2idx(x_text, word2idx, max_len)
  if target_words is not None and return_attr_target:
    target_idx = [word2idx[word] for word in target_words if word in word2idx]
    attr_target = np.zeros(x.shape)
    attr_target[np.isin(x, target_idx)] = 1.0
    attr_target = -1 * (attr_target - 1)
    return x, y, attr_target
  return x, y



In [ ]:
"""Train a Toxicity model using Keras."""

import datetime
import json
import os
import pickle

import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow.compat.v1 as tf
# import data_utils as utils

print('HELLO from model_tool')

DEFAULT_EMBEDDINGS_PATH = './glove.6B.100d.txt'
DEFAULT_MODEL_DIR = './'

DEFAULT_HPARAMS = {
    'max_sequence_length': 250,
    'max_num_words': 10000,
    'embedding_dim': 100,
    "seq_len": 10,
    'embedding_trainable': False,
    'learning_rate': 0.00005,
    'stop_early': True,
    'es_patience': 1,  # Only relevant if STOP_EARLY = True
    'es_min_delta': 0,  # Only relevant if STOP_EARLY = True
    'batch_size': 128,
    'epochs': 20,
    'dropout_rate': 0.3,
    'cnn_filter_sizes': [128, 128, 128],
    'cnn_kernel_sizes': [5, 5, 5],
    'cnn_pooling_sizes': [5, 5, 40],
    'verbose': True
}


def compute_auc(y_true, y_pred):
  try:
    return metrics.roc_auc_score(y_true, y_pred)
  except ValueError:
    return np.nan

### Model scoring


# Scoring these dataset for dozens of models actually takes non-trivial amounts
# of time, so we save the results as a CSV. The resulting CSV includes all the
# columns of the original dataset, and in addition has columns for each model,
# containing the model's scores.
def score_dataset(df, models, text_col):
  """Scores the dataset with each model and adds the scores as new columns."""
  for model in models:
    name = model.get_model_name()
    print('{} Scoring with {}...'.format(datetime.datetime.now(), name))
    df[name] = model.predict(df[text_col])


def load_maybe_score(models, orig_path, scored_path, postprocess_fn):
  """Return dataset specified by the given path and cache it with its scores."""
  if os.path.exists(scored_path):
    print('Using previously scored data:', scored_path)
    return pd.read_csv(scored_path)

  dataset = pd.read_csv(orig_path)
  postprocess_fn(dataset)
  score_dataset(dataset, models, 'text')
  print('Saving scores to:', scored_path)
  dataset.to_csv(scored_path)
  return dataset


def postprocess_madlibs(madlibs):
  """Modify madlibs data to have standard 'text' and 'label' columns."""
  # Native madlibs data uses 'Label' column with values 'BAD' and 'NOT_BAD'.
  # Replace with a bool.
  madlibs['label'] = madlibs['Label'] == 'BAD'
  madlibs.drop('Label', axis=1, inplace=True)
  madlibs.rename(columns={'Text': 'text'}, inplace=True)


def postprocess_wiki_dataset(wiki_data):
  """Modify Wikipedia dataset to have 'text' and 'label' columns."""
  wiki_data.rename(
      columns={
          'is_toxic': 'label',
          'comment': 'text'
      }, inplace=True)


class ToxModel():
  """Toxicity model."""

  def __init__(self,
               model_name=None,
               model_dir=DEFAULT_MODEL_DIR,
               embeddings_path=DEFAULT_EMBEDDINGS_PATH,
               hparams=None,
               typeArch="cnn",
               subtype=""):
    self.model_dir = model_dir
    self.embeddings_path = embeddings_path
    self.model_name = model_name
    self.model = None
    self.tokenizer = None
    self.embedding_matrix = None
    self.hparams = DEFAULT_HPARAMS.copy()
    self.typeArch = typeArch
    self.subtype = subtype
    self.num_classes = 2
    self.tokenize = False if self.typeArch == "cnn_attr" else True
    self.seq_len = DEFAULT_HPARAMS['seq_len']
    self.eager = False if self.typeArch == "cnn_attr" else True
    self.batch_size = self.hparams['batch_size']
    if hparams:
      self.update_hparams(hparams)
    if model_name:
      self.load_model_from_name(model_name)
    self.print_hparams()

  def print_hparams(self):
    print('Hyperparameters')
    print('---------------')
    for k, v in self.hparams.items():
      print('{}: {}'.format(k, v))
    print('')

  def update_hparams(self, new_hparams):
    self.hparams.update(new_hparams)

  def get_model_name(self):
    return self.model_name

  def save_hparams(self, model_name):
    self.hparams['model_name'] = model_name
    with open(
        os.path.join(self.model_dir, '%s_hparams.json' % self.model_name),
        'w') as f:
      json.dump(self.hparams, f, sort_keys=True)

  def load_model_from_name(self, model_name):
    """Load model given its name."""
    self.model = tf.keras.models.load_model(
        os.path.join(self.model_dir, '%s_model.h5' % model_name))
    self.tokenizer = pickle.load(
        open(
            os.path.join(self.model_dir, '%s_tokenizer.pkl' % model_name),
            'rb'))
    with open(
        os.path.join(self.model_dir, '%s_hparams.json' % self.model_name),
        'r') as f:
      self.hparams = json.load(f)

  def fit_and_save_tokenizer(self, texts):
    """Fits tokenizer on texts and pickles the tokenizer state."""
    self.tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=self.hparams['max_num_words'])
    self.tokenizer.fit_on_texts(texts)
    pickle.dump(
        self.tokenizer,
        open(
            os.path.join(self.model_dir, '%s_tokenizer.pkl' % self.model_name),
            'wb'))

  def prep_text(self, texts):
    """Turns text into into padded sequences.

    The tokenizer must be initialized before calling this method.

    Args:
      texts: Sequence of text strings.

    Returns:
      A tokenized and padded text sequence as a model input.
    """
    text_sequences = self.tokenizer.texts_to_sequences(texts)
    return tf.keras.preprocessing.sequence.pad_sequences(
        text_sequences, maxlen=self.hparams['max_sequence_length'])

  def load_embeddings(self):
    """Loads word embeddings."""
    embeddings_index = {}
    with open(self.embeddings_path) as f:
      for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

    self.embedding_matrix = np.zeros((len(self.tokenizer.word_index) + 1,
                                      self.hparams['embedding_dim']))
    num_words_in_embedding = 0
    for word, i in self.tokenizer.word_index.items():
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
        num_words_in_embedding += 1
        # words not found in embedding index will be all-zeros.
        self.embedding_matrix[i] = embedding_vector

  def train(self, training_data_path, validation_data_path, text_column,
            label_column, model_name):
    self.model_name = model_name
    self.save_hparams(model_name)

    train_data = pd.read_csv(training_data_path)
    valid_data = pd.read_csv(validation_data_path)
    train_data[text_column]=train_data[text_column].astype(str)
    valid_data[text_column]=valid_data[text_column].astype(str)


    if(self.tokenize):
      print('Fitting tokenizer...')
      self.fit_and_save_tokenizer(train_data[text_column])
      print('Tokenizer fitted!')

    print('Preparing data...')
    ##################
    if(self.typeArch == "cnn_attr"):
      target_words = []
      train_sep ="./wiki_test_sep.txt"
      val_sep = "./wiki_dev_sep.txt"
      self.seq_len = 100
      idx2word, word2idx = build_vocab(train_sep, 5, "./vocab.txt")
      train_text, train_labels, attr_target = preprocess(train_sep, self.seq_len, word2idx, target_words=target_words, return_attr_target=True, target_words_to_token=False)
      self.train_labels = train_labels
      self.baseline = get_all_pad(self.seq_len, word2idx)
      self.attr_target = attr_target
      self.word2idx = word2idx
      self.idx2word = idx2word
      valid_text, valid_labels = preprocess(val_sep, self.seq_len, word2idx, target_words=target_words)

    ##################
    else:
      train_text, train_labels = (self.prep_text(train_data[text_column]),
                                  tf.keras.utils.to_categorical(
                                      train_data[label_column]))
      valid_text, valid_labels = (self.prep_text(valid_data[text_column]),
                                tf.keras.utils.to_categorical(
                                    valid_data[label_column]))

    print('Data prepared!')

    if(self.tokenize):
      print('Loading embeddings...')
      self.load_embeddings()
      print('Embeddings loaded!')

    print('Building model graph...')
    self.build_model()
    print('Training model...')

    save_path = os.path.join(self.model_dir, '%s_model.h5' % self.model_name)
    callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
            save_path, save_best_only=True, verbose=self.hparams['verbose'])
    ]

    if self.hparams['stop_early']:
      callbacks.append(
          tf.keras.callbacks.EarlyStopping(
              min_delta=self.hparams['es_min_delta'],
              monitor='val_loss',
              patience=self.hparams['es_patience'],
              verbose=self.hparams['verbose'],
              mode='auto'))

    self.model.fit(
        train_text,
        train_labels,
        batch_size=self.batch_size,
        epochs=self.hparams['epochs'],
        validation_data=(valid_text, valid_labels),
        callbacks=callbacks,
        verbose=2)
    print('Model trained!')
    print('Best model saved to {}'.format(save_path))
    print('Loading best model from checkpoint...')
    self.model = tf.keras.models.load_model(save_path)
    print('Model loaded!')
    return self.model

  def train_shap(self, training_data_path, validation_data_path, text_column,
            label_column, model_name):
    train_data = pd.read_csv(training_data_path)
    valid_data = pd.read_csv(validation_data_path)
    train_data[text_column]=train_data[text_column].astype(str)
    valid_data[text_column]=valid_data[text_column].astype(str)
    self.fit_and_save_tokenizer(train_data[text_column])
    x_train = train_data[text_column]
    y_train = tf.keras.utils.to_categorical(
                                      train_data[label_column])
    x_test  = valid_data[text_column]
    y_test = tf.keras.utils.to_categorical(
                                      valid_data[label_column])


    # text_sequences = self.tokenizer.texts_to_sequences(texts)
    maxlen = self.hparams['max_sequence_length']

    x_train = tf.keras.preprocessing.sequence.pad_sequences(
        self.tokenizer.texts_to_sequences(x_train), maxlen=self.hparams['max_sequence_length'])
    x_test = tf.keras.preprocessing.sequence.pad_sequences(
        self.tokenizer.texts_to_sequences(x_test), maxlen=self.hparams['max_sequence_length'])

    model = Sequential()
    model.add(Embedding(20000, 128))
    model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2, activation='softmax'))
    rmsprop = tf.keras.optimizers.RMSprop(lr=self.hparams['learning_rate'])
    model.compile(
        loss='categorical_crossentropy', optimizer=rmsprop, metrics=['acc'])

    print('Train...')
    model.fit(x_train, y_train,
              batch_size=128,
              epochs=5,
              validation_data=(x_test, y_test))
    score, acc = model.evaluate(x_test, y_test,
                                batch_size=128)
    print('Test score:', score)
    print('Test accuracy:', acc)
    return model

  def buildRNN(self):
    sequence_input = tf.keras.layers.Input(
        shape=(self.hparams['max_sequence_length'],), dtype='int32')
    embedding_layer = tf.keras.layers.Embedding(
        len(self.tokenizer.word_index) + 1,
        self.hparams['embedding_dim'],
        weights=[self.embedding_matrix],
        input_length=self.hparams['max_sequence_length'],
        trainable=self.hparams['embedding_trainable'])

    embedded_sequences = embedding_layer(sequence_input)
    x = embedded_sequences
    if(self.subtype == ""):
      print("simple rnn")
      x = tf.keras.layers.SimpleRNN(128, input_shape=x.shape, return_sequences=False)(x)
    elif(self.subtype == "gru"):
      print("GRU")
      x = tf.keras.layers.GRU(128, input_shape=x.shape, return_sequences=False)(x)
    elif(self.subtype == "lstm"):
      print("LSTM")
      x = tf.keras.layers.LSTM(128, input_shape=x.shape, return_sequences=False)(x)

    x = tf.keras.layers.Dense(128, activation='relu')(x)
    preds = tf.keras.layers.Dense(2, activation='softmax')(x)

    rmsprop = tf.keras.optimizers.RMSprop(lr=self.hparams['learning_rate'])
    self.model = tf.keras.Model(sequence_input, preds)
    self.model.compile(
        loss='categorical_crossentropy', optimizer=rmsprop, metrics=['acc'])


  def buildCNN_attr(self):
    """Builds CNN ATTR model graph."""
    sequence_input = tf.keras.layers.Input(
        shape=(self.seq_len,), dtype='int32')
    # embedding_layer = tf.keras.layers.Embedding(
    #     len(self.tokenizer.word_index) + 1,
    #     self.hparams['embedding_dim'],
    #     weights=[self.embedding_matrix],
    #     input_length=self.hparams['max_sequence_length'],
    #     trainable=self.hparams['embedding_trainable'])

    # embedded_sequences = embedding_layer(sequence_input)
    # x = embedded_sequences

    ###PIG INTERPOLATION##########################################
    embedding_dim = 128
    init_embeddings = tf.random_uniform([len(self.idx2word),
                                           embedding_dim])
    x_emb = tf.nn.embedding_lookup(init_embeddings, sequence_input)
    x_emb_expand = tf.expand_dims(x_emb, -1)
    baseline_emb = tf.nn.embedding_lookup(init_embeddings, self.baseline)[0]

    scaled_inp = []
    self.steps = 50
    for i in range(1, self.steps+1):
      # print("baseline ", baseline_emb.shape, x.shape, sequence_input.shape)
      scaled_inp.append((float(i) / self.steps) * (x_emb-baseline_emb) + baseline_emb)
    inp_concat = tf.concat(scaled_inp, axis=0)
    inp_concat_stop = tf.stop_gradient(inp_concat)
    inp_concat_expand = tf.expand_dims(inp_concat_stop, -1)
    # x = inp_concat_expand
    ####END#####################################

    # for filter_size, kernel_size, pool_size in zip(
    #     self.hparams['cnn_filter_sizes'], self.hparams['cnn_kernel_sizes'],
    #     self.hparams['cnn_pooling_sizes']):
    #   x = self.build_conv_layer(x_emb_expand, filter_size, kernel_size, pool_size)

    pooled_outputs = []
    num_filters = 128
    filter_sizes = [2,3,4]
    pooled_outputs_scaled = []
    for i, filter_size in enumerate(filter_sizes):
      conv = tf.keras.layers.Conv2D(
                              filters=num_filters,
                              kernel_size=[filter_size, embedding_dim],
                              strides=(1, 1),
                              padding="VALID",
                              activation=tf.nn.relu,
                              name='conv_'+ str(i))
      conv = conv(x_emb_expand)
      pool = tf.keras.layers.MaxPooling2D(
                                      pool_size=[self.seq_len - filter_size + 1, 1],
                                      strides=(1, 1),
                                      padding="VALID")(conv)
      pooled_outputs.append(pool)


      conv_scaled = tf.keras.layers.Conv2D(
                                      filters=num_filters,
                                      kernel_size=[filter_size, embedding_dim],
                                      strides=(1, 1),
                                      padding="VALID",
                                      activation=tf.nn.relu,
                                      name='conv_'+ str(i))(inp_concat_expand)

      pool_scaled = tf.keras.layers.MaxPooling2D(
                                            pool_size=[self.seq_len - filter_size + 1, 1],
                                            strides=(1, 1),
                                            padding="VALID")(conv_scaled)
      pooled_outputs_scaled.append(pool_scaled)
    h_pool = tf.concat(pooled_outputs, 3)
    h_pool_scaled = tf.concat(pooled_outputs_scaled, 3)
    self.h_pool_flat = tf.reshape(h_pool, [-1, num_filters * len(filter_sizes)])
    self.h_pool_flat_scaled = tf.reshape(h_pool_scaled, [-1, num_filters * len(filter_sizes)])

    ####START#########
    drop_rate = 0.8
    mask = tf.nn.dropout(tf.ones_like(self.h_pool_flat), drop_rate)
    mask_scaled = tf.tile(mask, [self.steps, 1])
    h_drop = self.h_pool_flat * mask
    h_drop_scaled = self.h_pool_flat_scaled * mask_scaled

    logits = tf.keras.layers.Dense(self.num_classes, activation=None)(h_drop)
    logits_scaled = tf.keras.layers.Dense(self.num_classes, activation=None)(h_drop_scaled)
    softmax = tf.nn.softmax(logits)
    preds = softmax
    softmax_scaled = tf.nn.softmax(logits_scaled)
    ####END########


    #####PIG############################################
    self.grad = tf.gradients(softmax_scaled[:, 1], inp_concat_stop)[0]

    grad_reshape = tf.reshape(self.grad, [self.steps, -1, self.seq_len, embedding_dim])
    avg_grad = tf.reduce_mean(grad_reshape, axis=0)
    intergrated_grad = avg_grad * (x_emb - baseline_emb)
    self.tmp = intergrated_grad
    self.sum_intergrated_grad = tf.reduce_sum(intergrated_grad, axis=2)
    self.target_attribution = tf.multiply(self.sum_intergrated_grad, self.attr_target)

    #######END##########################################

    rmsprop = tf.keras.optimizers.Adam(learning_rate=self.hparams['learning_rate'])
    print("seq/pred lengths", sequence_input.shape, preds.shape)
    self.model = tf.keras.Model(sequence_input, preds)


    def myLoss(label, pred):
      attr_loss_weight=100000000000
      labels = tf.one_hot(tf.cast(label, tf.int32), depth=self.num_classes)
      cross_entropy = - tf.reduce_sum(tf.multiply(labels, pred), reduction_indices=1)
      clf_loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')
      attr_loss = tf.losses.mean_squared_error(self.target_attribution, self.sum_intergrated_grad) * attr_loss_weight
      print("Attr loss", attr_loss.shape, clf_loss.shape)
      return clf_loss + attr_loss
    self.model.compile(
        loss=myLoss, optimizer=rmsprop, metrics=['acc'], run_eagerly=self.eager)

  def buildCNN(self):
    """Builds model graph."""
    sequence_input = tf.keras.layers.Input(
        shape=(self.hparams['max_sequence_length'],), dtype='int32')
    embedding_layer = tf.keras.layers.Embedding(
        len(self.tokenizer.word_index) + 1,
        self.hparams['embedding_dim'],
        weights=[self.embedding_matrix],
        input_length=self.hparams['max_sequence_length'],
        trainable=self.hparams['embedding_trainable'])

    embedded_sequences = embedding_layer(sequence_input)
    x = embedded_sequences
    for filter_size, kernel_size, pool_size in zip(
        self.hparams['cnn_filter_sizes'], self.hparams['cnn_kernel_sizes'],
        self.hparams['cnn_pooling_sizes']):
      x = self.build_conv_layer(x, filter_size, kernel_size, pool_size)

    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(self.hparams['dropout_rate'])(x)
    # TODO(nthain): Parametrize the number and size of fully connected layers
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    preds = tf.keras.layers.Dense(2, activation='softmax')(x)

    rmsprop = tf.keras.optimizers.RMSprop(lr=self.hparams['learning_rate'])
    print("seq/pred lengths", sequence_input.shape, preds.shape)
    self.model = tf.keras.Model(sequence_input, preds)
    self.model.compile(
        loss='categorical_crossentropy', optimizer=rmsprop, metrics=['acc'])

  def build_model(self):
    if(self.typeArch == "cnn"):
      print("USING", "CNN" )
      self.buildCNN()
    elif(self.typeArch == "rnn"):
      print("USING", "RNN" )
      self.buildRNN()
    elif(self.typeArch == "cnn_attr"):
      print("USING", "CNN ATTR" )
      # tf.compat.v1.disable_eager_execution()
      self.buildCNN_attr()
      # tf.compat.v1.enable_eager_execution()

  def build_conv_layer(self, input_tensor, filter_size, kernel_size, pool_size):
    output = tf.keras.layers.Conv1D(
        filter_size, kernel_size, activation='relu', padding='same')(
            input_tensor)
    if pool_size:
      output = tf.keras.layers.MaxPool1D(pool_size, padding='same')(output)
    else:
      output = tf.keras.layers.GlobalMaxPool1D()(output)
    return output

  def predict(self, texts):
    """Returns model predictions on texts."""
    data = self.prep_text(texts)
    return self.model.predict(data)[:, 1]

  def score_auc(self, texts, labels):
    preds = self.predict(texts)
    return compute_auc(labels, preds)

  def summary(self):
    return self.model.summary()


HELLO from model_tool


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import pandas as pd
hparams = {'epochs': 4}
SPLITS = ['train', 'dev', 'test']

wiki = {}
# debias = {}
# random = {}
noisy = {}
for split in SPLITS:
    wiki[split] = './wiki_%s.csv' % split
    noisy[split] = './wiki_noised_%s.csv' % split
    # random[split] = '../data/wiki_debias_random_%s.csv' % split

In [ ]:
# MODEL_NAME = 'cnn_wiki_tox_v3'
# wiki_model = ToxModel(hparams=hparams)
# wiki_model.train(wiki['train'], wiki['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)
typeData = noisy
MODEL_NAME = 'cnn_wiki_noised_tox_v3'
wiki_model = ToxModel(hparams=hparams)
cnn_model = wiki_model.train(typeData['train'], typeData['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

Hyperparameters
---------------
max_sequence_length: 250
max_num_words: 10000
embedding_dim: 100
seq_len: 10
embedding_trainable: False
learning_rate: 5e-05
stop_early: True
es_patience: 1
es_min_delta: 0
batch_size: 128
epochs: 4
dropout_rate: 0.3
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
cnn_pooling_sizes: [5, 5, 40]
verbose: True

Fitting tokenizer...
Tokenizer fitted!
Preparing data...


KeyboardInterrupt: ignored

In [ ]:
# MODEL_NAME = 'cnn_wiki_tox_v3'
# wiki_model = ToxModel(hparams=hparams)
# wiki_model.train(wiki['train'], wiki['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)
typeData = noisy
MODEL_NAME = 'shap_wiki_noised_tox_v3'
wiki_model = ToxModel(hparams=hparams)
shap_model = wiki_model.train_shap(typeData['train'], typeData['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

Hyperparameters
---------------
max_sequence_length: 250
max_num_words: 10000
embedding_dim: 100
seq_len: 10
embedding_trainable: False
learning_rate: 5e-05
stop_early: True
es_patience: 1
es_min_delta: 0
batch_size: 128
epochs: 4
dropout_rate: 0.3
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
cnn_pooling_sizes: [5, 5, 40]
verbose: True



/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/rmsprop.py:144: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Train...
Epoch 1/5
748/748 [==============================] - 917s 1s/step - loss: 0.3243 - acc: 0.9024 - val_loss: 0.2638 - val_acc: 0.9051
Epoch 2/5
748/748 [==============================] - 920s 1s/step - loss: 0.2342 - acc: 0.9084 - val_loss: 0.2156 - val_acc: 0.9188
Epoch 3/5
748/748 [==============================] - 919s 1s/step - loss: 0.2014 - acc: 0.9224 - val_loss: 0.1948 - val_acc: 0.9273
Epoch 4/5
748/748 [==============================] - 928s 1s/step - loss: 0.1843 - acc: 0.9320 - val_loss: 0.1826 - val_acc: 0.9334
Epoch 5/5
251/251 [==============================] - 19s 75ms/step - loss: 0.1778 - acc: 0.9383
Test score: 0.17781531810760498
Test accuracy: 0.9383092522621155


In [ ]:
with open('./shap.pkl', 'wb') as f:
    pickle.dump(shap_model, f)

In [ ]:
train_data = pd.read_csv(typeData['train'])
valid_data = pd.read_csv(typeData['dev'])
train_data['comment']=train_data['comment'].astype(str)
valid_data['comment']=valid_data['comment'].astype(str)
tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=10000)
tokenizer.fit_on_texts(train_data['comment'])

x_train = train_data['comment']
y_train = tf.keras.utils.to_categorical(
                                  train_data['is_toxic'])
x_test  = valid_data['comment']
y_test = tf.keras.utils.to_categorical(
                                  valid_data['is_toxic'])


# text_sequences = self.tokenizer.texts_to_sequences(texts)

x_train = tf.keras.preprocessing.sequence.pad_sequences(
    tokenizer.texts_to_sequences(x_train), maxlen=250)
x_test = tf.keras.preprocessing.sequence.pad_sequences(
    tokenizer.texts_to_sequences(x_test), maxlen=250)

In [ ]:
import numpy as np
import pandas as pd
import shap

# select a set of background examples to take an expectation over
# x_train, y_train, x_test, y_test = m.getData(typeData['train'], typeData['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)
# we use the first 100 training examples as our background dataset to integrate over
explainer = shap.DeepExplainer(shap_model, x_train[:100])

# explain the first 10 predictions
# explaining each prediction requires 2 * background dataset size runs
print(x_test[:10])
shap_values = explainer.shap_values(x_train[:10])
# init the JS visualization code
shap.initjs()

# transform the indexes to words
import numpy as np
words = imdb.get_word_index()
num2word = {}
for w in words.keys():
    num2word[words[w]] = w
x_test_words = np.stack([np.array(list(map(lambda x: num2word.get(x, "NONE"), x_test[i]))) for i in range(10)])

# plot the explanation of the first prediction
# Note the model is "multi-output" because it is rank-2 but only has one column
shap.force_plot(explainer.expected_value[0], shap_values[0][0], x_test_words[0])


Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


[[   1 1473   29 ... 2399   33    1]
 [ 614    1    2 ...    1    1    1]
 [   0    0    0 ...  294 1077    1]
 ...
 [   0    0    0 ... 1414   13   21]
 [   0    0    0 ...    1  439    6]
 [   0    0    0 ...  113    1   16]]


TypeError: ignored

In [ ]:
MODEL_NAME = 'RNN_noised_wiki_tox'
typeData = noisy
wiki_model = ToxModel(hparams=hparams, typeArch="rnn")
wiki_model.train(typeData['train'], typeData['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

In [ ]:
# MODEL_NAME = 'CNN-attr_wiki_tox'
# wiki_model = ToxModel(hparams=hparams, typeArch="cnn_attr")
# wiki_model.train(wiki['train'], wiki['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

In [ ]:
MODEL_NAME = 'RNN-LSTM_wiki_tox'
typeData = wiki
wiki_model = ToxModel(hparams=hparams, typeArch="rnn", subtype="lstm")
wiki_model.train(typeData['train'], typeData['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

Hyperparameters
---------------
max_sequence_length: 250
max_num_words: 10000
embedding_dim: 100
seq_len: 10
embedding_trainable: False
learning_rate: 5e-05
stop_early: True
es_patience: 1
es_min_delta: 0
batch_size: 128
epochs: 4
dropout_rate: 0.3
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
cnn_pooling_sizes: [5, 5, 40]
verbose: True

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
USING RNN
LSTM
Training model...
Epoch 1/4


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/rmsprop.py:144: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)



Epoch 1: val_loss improved from inf to 0.18536, saving model to ./RNN-LSTM_wiki_tox_model.h5
748/748 - 1179s - loss: 0.2259 - acc: 0.9231 - val_loss: 0.1854 - val_acc: 0.9317 - 1179s/epoch - 2s/step
Epoch 2/4


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 0.18536 to 0.16700, saving model to ./RNN-LSTM_wiki_tox_model.h5
748/748 - 1268s - loss: 0.1695 - acc: 0.9387 - val_loss: 0.1670 - val_acc: 0.9411 - 1268s/epoch - 2s/step
Epoch 3/4

Epoch 3: val_loss improved from 0.16700 to 0.15653, saving model to ./RNN-LSTM_wiki_tox_model.h5
748/748 - 1207s - loss: 0.1585 - acc: 0.9424 - val_loss: 0.1565 - val_acc: 0.9441 - 1207s/epoch - 2s/step
Epoch 4/4

Epoch 4: val_loss improved from 0.15653 to 0.14984, saving model to ./RNN-LSTM_wiki_tox_model.h5
748/748 - 1221s - loss: 0.1524 - acc: 0.9442 - val_loss: 0.1498 - val_acc: 0.9442 - 1221s/epoch - 2s/step
Model trained!
Best model saved to ./RNN-LSTM_wiki_tox_model.h5
Loading best model from checkpoint...
Model loaded!


In [ ]:
MODEL_NAME = 'RNN-LSTM_noisy_wiki_tox'
typeData = noisy
wiki_model = ToxModel(hparams=hparams, typeArch="rnn", subtype="lstm")
wiki_model.train(typeData['train'], typeData['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

Hyperparameters
---------------
max_sequence_length: 250
max_num_words: 10000
embedding_dim: 100
seq_len: 10
embedding_trainable: False
learning_rate: 5e-05
stop_early: True
es_patience: 1
es_min_delta: 0
batch_size: 128
epochs: 4
dropout_rate: 0.3
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
cnn_pooling_sizes: [5, 5, 40]
verbose: True

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
USING RNN
LSTM
Training model...
Epoch 1/4


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/rmsprop.py:144: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)



Epoch 1: val_loss improved from inf to 0.20581, saving model to ./RNN-LSTM_noisy_wiki_tox_model.h5
748/748 - 825s - loss: 0.2533 - acc: 0.9150 - val_loss: 0.2058 - val_acc: 0.9271 - 825s/epoch - 1s/step
Epoch 2/4


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 0.20581 to 0.18868, saving model to ./RNN-LSTM_noisy_wiki_tox_model.h5
748/748 - 760s - loss: 0.1992 - acc: 0.9292 - val_loss: 0.1887 - val_acc: 0.9344 - 760s/epoch - 1s/step
Epoch 3/4

Epoch 3: val_loss improved from 0.18868 to 0.18266, saving model to ./RNN-LSTM_noisy_wiki_tox_model.h5
748/748 - 757s - loss: 0.1894 - acc: 0.9321 - val_loss: 0.1827 - val_acc: 0.9356 - 757s/epoch - 1s/step
Epoch 4/4

Epoch 4: val_loss improved from 0.18266 to 0.17985, saving model to ./RNN-LSTM_noisy_wiki_tox_model.h5
748/748 - 778s - loss: 0.1842 - acc: 0.9343 - val_loss: 0.1799 - val_acc: 0.9362 - 778s/epoch - 1s/step


Model trained!
Best model saved to ./RNN-LSTM_noisy_wiki_tox_model.h5
Loading best model from checkpoint...
Model loaded!


In [ ]:
MODEL_NAME = 'CNN_wiki_tox'
typeData = wiki
wiki_model = ToxModel(hparams=hparams, typeArch="cnn")
wiki_model.train(typeData['train'], typeData['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

Hyperparameters
---------------
max_sequence_length: 250
max_num_words: 10000
embedding_dim: 100
seq_len: 10
embedding_trainable: False
learning_rate: 5e-05
stop_early: True
es_patience: 1
es_min_delta: 0
batch_size: 128
epochs: 4
dropout_rate: 0.3
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
cnn_pooling_sizes: [5, 5, 40]
verbose: True

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
USING CNN


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/rmsprop.py:144: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


seq/pred lengths (None, 250) (None, 2)
Training model...
Epoch 1/4

Epoch 1: val_loss improved from inf to 0.17557, saving model to ./CNN_wiki_tox_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


748/748 - 15s - loss: 0.2383 - acc: 0.9154 - val_loss: 0.1756 - val_acc: 0.9361 - 15s/epoch - 20ms/step
Epoch 2/4

Epoch 2: val_loss improved from 0.17557 to 0.15794, saving model to ./CNN_wiki_tox_model.h5
748/748 - 8s - loss: 0.1681 - acc: 0.9380 - val_loss: 0.1579 - val_acc: 0.9426 - 8s/epoch - 10ms/step
Epoch 3/4

Epoch 3: val_loss improved from 0.15794 to 0.14523, saving model to ./CNN_wiki_tox_model.h5
748/748 - 7s - loss: 0.1495 - acc: 0.9441 - val_loss: 0.1452 - val_acc: 0.9471 - 7s/epoch - 10ms/step
Epoch 4/4

Epoch 4: val_loss did not improve from 0.14523
748/748 - 8s - loss: 0.1373 - acc: 0.9490 - val_loss: 0.1476 - val_acc: 0.9436 - 8s/epoch - 10ms/step
Epoch 4: early stopping
Model trained!
Best model saved to ./CNN_wiki_tox_model.h5
Loading best model from checkpoint...
Model loaded!


In [ ]:
MODEL_NAME = 'RNN-GRU_noisy_wiki_tox'
typeData = noisy
wiki_model = ToxModel(hparams=hparams, typeArch="rnn", subtype="gru")
wiki_model.train(typeData['train'], typeData['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

Hyperparameters
---------------
max_sequence_length: 250
max_num_words: 10000
embedding_dim: 100
seq_len: 10
embedding_trainable: False
learning_rate: 5e-05
stop_early: True
es_patience: 1
es_min_delta: 0
batch_size: 128
epochs: 4
dropout_rate: 0.3
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
cnn_pooling_sizes: [5, 5, 40]
verbose: True

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
USING RNN
GRU
Training model...
Epoch 1/4


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/rmsprop.py:144: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)



Epoch 1: val_loss improved from inf to 0.21206, saving model to ./RNN-GRU_noisy_wiki_tox_model.h5
748/748 - 621s - loss: 0.2735 - acc: 0.9073 - val_loss: 0.2121 - val_acc: 0.9296 - 621s/epoch - 831ms/step
Epoch 2/4


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 0.21206 to 0.18744, saving model to ./RNN-GRU_noisy_wiki_tox_model.h5
748/748 - 627s - loss: 0.1969 - acc: 0.9303 - val_loss: 0.1874 - val_acc: 0.9347 - 627s/epoch - 838ms/step
Epoch 3/4

Epoch 3: val_loss did not improve from 0.18744
748/748 - 639s - loss: 0.1859 - acc: 0.9335 - val_loss: 0.1895 - val_acc: 0.9327 - 639s/epoch - 855ms/step
Epoch 3: early stopping
Model trained!
Best model saved to ./RNN-GRU_noisy_wiki_tox_model.h5
Loading best model from checkpoint...


Model loaded!


In [ ]:
MODEL_NAME = 'RNN-GRU_wiki_tox'
typeData = wiki
wiki_model = ToxModel(hparams=hparams, typeArch="rnn", subtype="gru")
wiki_model.train(typeData['train'], typeData['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

Hyperparameters
---------------
max_sequence_length: 250
max_num_words: 10000
embedding_dim: 100
seq_len: 10
embedding_trainable: False
learning_rate: 5e-05
stop_early: True
es_patience: 1
es_min_delta: 0
batch_size: 128
epochs: 4
dropout_rate: 0.3
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
cnn_pooling_sizes: [5, 5, 40]
verbose: True

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
USING RNN
GRU
Training model...
Epoch 1/4


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/rmsprop.py:144: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)



Epoch 1: val_loss improved from inf to 0.18369, saving model to ./RNN-GRU_wiki_tox_model.h5
748/748 - 621s - loss: 0.2405 - acc: 0.9213 - val_loss: 0.1837 - val_acc: 0.9362 - 621s/epoch - 830ms/step
Epoch 2/4


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 0.18369 to 0.15777, saving model to ./RNN-GRU_wiki_tox_model.h5
748/748 - 637s - loss: 0.1707 - acc: 0.9395 - val_loss: 0.1578 - val_acc: 0.9424 - 637s/epoch - 852ms/step
Epoch 3/4

Epoch 3: val_loss improved from 0.15777 to 0.14881, saving model to ./RNN-GRU_wiki_tox_model.h5
748/748 - 646s - loss: 0.1568 - acc: 0.9431 - val_loss: 0.1488 - val_acc: 0.9450 - 646s/epoch - 864ms/step
Epoch 4/4

Epoch 4: val_loss improved from 0.14881 to 0.14495, saving model to ./RNN-GRU_wiki_tox_model.h5
748/748 - 623s - loss: 0.1487 - acc: 0.9456 - val_loss: 0.1449 - val_acc: 0.9463 - 623s/epoch - 833ms/step


Model trained!
Best model saved to ./RNN-GRU_wiki_tox_model.h5
Loading best model from checkpoint...
Model loaded!
